In [15]:
import pandas as pd
import numpy as np
from keras_bert import get_model, gen_batch_inputs, get_base_dict
import keras
import pickle

In [16]:
pickle_in = open("token_dict.pickle","rb")
token_dict = pickle.load(pickle_in)
token_dict_rev = {v: k for k, v in token_dict.items()}

In [17]:
largest_seq = 420

model = get_model(
    token_num=len(token_dict),
    head_num=2, # Should be a factor of embed_dim
    transformer_num=2,
    embed_dim=256,
    feed_forward_dim=100,
    seq_len=largest_seq,
    pos_num=largest_seq,
    dropout_rate=0.05,
)

model.load_weights("tinybertweights.hdf5")

In [30]:
import codecs
bsz = 1
MASK = ['<MASK>']
CLS = ['<CLS>']
SEP = ['<SEP>']
UNK = ['<UNK>']

#cleared, abnormalities
tokens = CLS + 'ON 01/04/2019 at 01:29 The Control Room received alarm window 5B02D , PPS TROUBLE / GND ,\
and RJ point SBYS36, Plant Protection System Channel D Ground. The alarm locked in for 45 minutes \
then '.lower().split() + MASK +\
'. The alarm response was followed and no '.lower().split() + MASK +  ' were identified'.lower().split() + SEP + ' The ground alarm light on the\
front of PPS Cabinet D is ON which is normal , meaning there is no ground indicated '.lower().split() + SEP + ' Wrote CR to\
request support from I&C '.lower().split() + SEP

tokens = CLS + ' during ist testing, valve '.split() + MASK + ' failed reference '.split() + MASK + SEP

tokens = CLS + ' cooling fan , '.split() + MASK + ' failed '.split() + SEP


tokens

['<CLS>', 'cooling', 'fan', ',', '<MASK>', 'failed', '<SEP>']

In [31]:
def token_lookup(token):
    try:
        x = token_dict[token]
        return x
    except KeyError:
        return token_dict['<UNK>']
    
token_input = np.asarray([[token_lookup(token) for token in tokens] + [0] * (largest_seq - len(tokens)) for i in range(bsz)])
seg_input = np.asarray([[0] * len(tokens) + [0] * (largest_seq - len(tokens)) for i in range(bsz)])
mask_input = np.asarray([[0] + [0] * (largest_seq - 1) for i in range(bsz)])

for i, x in enumerate(tokens):
    if x == '<MASK>':
        mask_input[0][i] = 1


In [32]:
print('input:', token_input[0][:len(tokens)])
predicts = model.predict([token_input, seg_input, mask_input])[0]
predicts = np.argmax(predicts, axis=-1)
print('predi:', predicts[0][:len(tokens)])
print('words:', list(map(lambda x: token_dict_rev[x], predicts[0][1:len(tokens)])))

input: [   2 4200 6071   28    4  994    3]
predi: [  76 4200 6071   28 4253  994   76]
words: ['cooling', 'fan', ',', 'breaker', 'failed', 'to']


In [33]:
list(token_dict)[:5]

['', '<UNK>', '<CLS>', '<SEP>', '<MASK>']